<a href="https://colab.research.google.com/github/ImPeekaboo/mytools/blob/main/libtorrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Setup**

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=4>**Install rclone**</font>
build_version = "stable" #@param ["stable", "beta"]
automatically_clear_cell_output = True  # @param{type: "boolean"}
# ================================================================ #

import os
import IPython
from IPython.display import clear_output


if build_version == "stable":
    !curl https://rclone.org/install.sh | sudo bash
else:
    !curl https://rclone.org/install.sh | sudo bash -s beta

!sudo apt-get -y install fuse3

try:
    os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
    pass

if automatically_clear_cell_output is True:
    clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Upload Configuration File**</font>
# @markdown <p>If you already have an rclone configuration file, you can upload it by running this cell.</p>
# ================================================================ #

from google.colab import files
from IPython.display import clear_output
import shutil
import os

uploaded = files.upload()

destination_directory = '/root/.config/rclone'

for filename in uploaded.keys():
    shutil.move(filename, destination_directory + '/' + filename)

clear_output()

print(filename, 'success uploaded to', destination_directory)

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Mount Google Drive**</font>
# @markdown <p>Run this if you don't want to upload rclone config, but you still need run ***Install rclone*** above to use rclone transfer to transfer your files to your google drive.</p>
# ================================================================ #

from google.colab import drive
drive.mount('/content/drive')

####**Libtorrent**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**libtorrent for torrent file**</font>
# @markdown <p>Run this cell to use libtorrent with torrent file.</p>
# ================================================================ #

import os

if os.path.exists("/content/downloads"):
  pass
else:
  os.mkdir("/content/downloads")

import importlib
from IPython.display import clear_output

if importlib.util.find_spec("libtorrent") is None:
    !pip install libtorrent
    clear_output()

import libtorrent as lt
import time
import sys
from google.colab import files

ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

uploaded = files.upload()
uploaded_file_path = list(uploaded.keys())[0]
clear_output()

info = lt.torrent_info(uploaded_file_path)
h = ses.add_torrent({'ti': info, 'save_path': '/content/downloads'})
s = h.status()

print('start downloading', s.name)

!rm {uploaded_file_path}

while (not s.is_seeding):
    s = h.status()

    print('\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
        s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
        s.num_peers, s.state), end=' ')


# this optional, you can include this too
#    alerts = ses.pop_alerts()
#    for a in alerts:
#        if a.category() & lt.alert.category_t.error_notification:
#            print(a)

    sys.stdout.flush()

    time.sleep(1)

print(h.status().name, 'complete')

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**libtorrent for magnet link**</font>
# @markdown <p>Run this cell to use libtorrent with magnet link.</p>
# ================================================================ #

from IPython.display import clear_output
import os
import importlib

if os.path.exists("/content/downloads"):
  pass
else:
  os.mkdir("/content/downloads")

if importlib.util.find_spec("libtorrent") is None:
    !pip install libtorrent
    clear_output()

import libtorrent as lt
import time
import sys
import urllib.parse

ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

magnet_link = input('Paste Magnet Link here:'\n)
clear_output()

params = lt.add_torrent_params()
params.save_path = '/content/downloads'

info = lt.parse_magnet_uri(magnet_link)
info_hash = info.info_hashes
params.info_hashes = info_hash

h = ses.add_torrent(params)
s = h.status()

parsed_magnet_link = urllib.parse.urlparse(magnet_link)
torrent_name = urllib.parse.unquote(parsed_magnet_link.query.split('&dn=')[1].split('&')[0])

print('start downloading', torrent_name)

while (not s.is_seeding):
    s = h.status()

    print('\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
        s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
        s.num_peers, s.state), end=' ')

#    alerts = ses.pop_alerts()
#    for a in alerts:
#        if a.category() & lt.alert.category_t.error_notification:
#            print(a)

    sys.stdout.flush()

    time.sleep(1)

print(h.status().name, 'complete')

####**Rclone & Input Command**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Transfer Files with Rclone**</font>
# @markdown <p>Use ***copy*** mode for files from your path and ***copyurl*** mode for direct link download, fill ***source*** with link or patch source, ***dest*** with your destiny path.</p>
mode = "copy" #@param ["copy", "copyurl"]
source = "/content/archive-result" #@param {type:"string"}
dest = "" #@param {type:"string"}
# ================================================================ #

# command
if mode == "copyurl":
    command = f"rclone {mode} -v --stats 3 --stats-one-line --onedrive-chunk-size 40M --multi-thread-cutoff 100M --multi-thread-streams 16 --transfers 15 -a \"{source}\" \"{dest}\" --user-agent 'ISV|rclone.org|rclone/v1.62.2'"
else:
    command = f"rclone {mode} -v --stats 3 --stats-one-line --onedrive-chunk-size 40M --multi-thread-cutoff 100M --multi-thread-streams 16 --transfers 15 \"{source}\" \"{dest}\" --user-agent 'ISV|rclone.org|rclone/v1.62.2'"

!{command}

In [ ]:
# input your custom commands here